# Repeat Experiment 
## import packages

In [ ]:
import warnings
import torch

warnings.filterwarnings("ignore")

In [ ]:
from tcnn.utils.experiment.repeat import repeat_experiment

In [ ]:
from os.path import join
import torch

dataset_name = "fashionMNIST"
config = {
    "order": "04",
    "task": "multiclass",
    "data": {
        "batch_size": 64,
        "size": 32,
    },
    "network": {"input_channels": 1, "linear_size": 400, "num_classes": 10},
    "train": {
        "criterion": torch.nn.CrossEntropyLoss(),
        "checkpoint_save_dir": "checkpoints",
        "epochs": 50,
        "device": "cuda" if torch.cuda.is_available() else "cpu",
    },
    "repeat": {"num_experiments": 5, 
               "epochs_per_experiemnt": 100,
               "log_save_dir": "logs",
               },
}

In [ ]:
ORDER = config["order"]
SIZE = config["data"]["size"]
TASK = config["task"]
NUM_CLASSES = config["network"]["num_classes"]
EPOCHS = config["train"]["epochs"]
BATCH_SIZE = config["data"]["batch_size"]
INPUT_CHANNEL = config["network"]["input_channels"]
LINEAR_SIZE = config["network"]["linear_size"]
CRITERION = config["train"]["criterion"]
DEVICE = config["train"]["device"]
NUM_EXPERIMENTS = config["repeat"]["num_experiments"]
EPOCHS_PER_EXPERIMENT = config["repeat"]["epochs_per_experiemnt"]
experiment_name = (
    f"repeat-{ORDER}-{dataset_name}-{SIZE}-{NUM_EXPERIMENTS}-{EPOCHS_PER_EXPERIMENT}"
)

In [ ]:
checkpoint_save_dir = join(config["train"]["checkpoint_save_dir"], experiment_name)
log_save_dir = join(config["repeat"]["log_save_dir"])

## get dataset

In [ ]:
from torchvision import datasets, transforms

train_dataset = datasets.FashionMNIST(
    "./data",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]),
)

test_dataset = datasets.FashionMNIST(
    "./data",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]),
)

## reset dataloader

In [ ]:
def reset_dataloader(dataset, batch_size, shuffle=True):
    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
    )

## import models

In [ ]:
import lenet_models

model_dict = lenet_models.get_constant_model_dict(INPUT_CHANNEL, NUM_CLASSES, LINEAR_SIZE)

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR


def def_optimizer(model):
    return torch.optim.Adam(model.parameters(), lr=0.001)


def def_scheduler(optimizer):
    return ExponentialLR(optimizer, gamma=0.9)

## define result dict

In [ ]:
results = dict()

## repeat experiment

In [ ]:
for model_name, model in model_dict.items():
    print("Traing model: ", model_name)
    results[model_name] = repeat_experiment(
        model,
        [train_dataset, test_dataset],
        reset_dataloader,
        BATCH_SIZE,
        CRITERION,
        def_optimizer,
        def_scheduler,
        NUM_EXPERIMENTS,
        EPOCHS_PER_EXPERIMENT,
        experiment_name + "-" + model_name,
        checkpoint_save=True,
        checkpoint_save_dir=checkpoint_save_dir,
    )
    with torch.no_grad():
        torch.cuda.empty_cache()
    print("***" * 10 + f"{model_name} done" + "***" * 10)

In [ ]:
from tcnn.utils.experiment.log import show_repeat_result, save_result

show_repeat_result(results)
save_result(results, save_name=experiment_name)

In [ ]:
from tcnn.utils.experiment.plot import plot_experiment_errorbar

plot_experiment_errorbar(
    results, metric_key="accuracy", baseline_key="lenet", ylabel="Accuracy"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="accuracy", baseline_key="lenet_relu", ylabel="Accuracy"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="auc_score", baseline_key="lenet", ylabel="AUC Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="auc_score", baseline_key="lenet_relu", ylabel="AUC Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="f1", baseline_key="lenet", ylabel="F1 Score"
)

In [ ]:
plot_experiment_errorbar(
    results, metric_key="f1", baseline_key="lenet_relu", ylabel="F1 Score"
)

In [ ]:
import torch

with torch.no_grad():
    torch.cuda.empty_cache()